<h3 style="color: lightgreen;">Import SQL</h3>
sqlite3 is a python module that helps us interact with databases using SQL langauge

<a href="https://1drv.ms/u/s!AoYG-9xuwdAJgU6ze0McmNt7cS4x?e=GYZY2g">click here to see my notes on this...</a>

In [1]:
import sqlite3

<h3 style="color: lightgreen;">Create a Cursor Object</h3>

sqlite3 is a module that returns an object that is also called "sqlite3". Out the gate, this comes with a method called <span style="color: orange;">connect()</span>, which takes the path to a specific database file as it's argument. It then returns a connection object (we've decided to name this "CONN" but name it whatever you like). 

CONN has a method called <span style="color: orange;">cursor()</span>, which returns a "cursor" object. Again, this is an object within the big hierchy of classes in the sqlite3 module, and so it has some prebuilt methods already in it (<a href="https://1drv.ms/u/s!AoYG-9xuwdAJgU6ze0McmNt7cS4x?e=GYZY2g">refer again to my notes on this for some more detail</a>)

In [4]:
CONN = sqlite3.connect('dogs.db')
CURSOR = CONN.cursor()

<h3 style="color: lightgreen;">Dog Class</h3>

The Dog class is what we're using to model a dog. Every instance of a dog has a name, a breed, and an id

In [6]:
class Dog:
    def __init__(self, name, breed, id=None): 
        self.name = name
        self.breed = breed
        self.id = id
        
    
        

<h3 style="color: lightgreen;">Creating a Table</h3>

The OOP counterpart to a database table is a class. Each row in this table corresponds to a specific instance of that class. And each column corresponds to a specific attribute of that class. 

The cursor object (which is the instance of the Cursor class that we instantiated earlier), has a method called <span style="color: orange;">execute()</span>, which will execute any SQL statement we pass into it. 

In [15]:
def create_table(): 
        CURSOR.execute('''
            CREATE TABLE IF NOT EXISTS dogs (
                id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                breed TEXT NOT NULL
            )
        ''')
        CONN.commit()

<h3 style="color: lightgreen;">Drop Table</h3>

'DROP TABLE IF EXISTS dogs' is the SQL statement for dropping a table. In the statement there is conditional logic that checks to see if a "dogs" table already exists. 

In the given code, <span style="color: orange;">commit()</span> is used to commit the changes made to the database after executing the SQL statement that creates a new table dogs with three columns id, name, and breed. Calling commit is comparable to calling POST as a client. We can write all the SQL we want, but if we're not committing via the <span style="color: orange;">commit()</span> method, then our changes aren't being sent to the database. Be sure to call the commit method on the CONN object. 






In [18]:
def drop_table(): 
        CURSOR.execute('DROP TABLE IF EXISTS dogs')
        CONN.commit()

<h3 style="color: lightgreen;">Save</h3>

Similar setup as the drop_table function, except using the SQL statements for inserting into the database (just think SAVE = INSERT. We're persisting the data in this method)

Note the question marks '?'. These are placeholders which is a best practice for SQL. It's a feature specific to sql that helps prevent SQL injection (nasty cyber hack). If we did string interpolation instead, then we'd be making our database vulnerable to malicious attacks. 

In [9]:
def save(self): 
    CURSOR.execute('INSERT INTO dogs (name, breed) VALUES (?, ?)', (self.name, self.breed))
    CONN.commit()
    self.id = CURSOR.lastrowid

<h3 style="color: lightgreen;">Create</h3>

The next few methods are class methods, denoted by the @classmethod decorators. When a class method is called, we are calling a method that will affect all of the instances of that class. Just remember that we need to pass cls as an argument. This is a reserved keyword much like the self arguments when referring to just the instances of the class

In [10]:
@classmethod
def create(cls, name, breed):
    dog = cls(name, breed)
    dog.save()
    return dog

In [11]:
# This class method is used to create a new Dog instance from a row in the database.
# It takes a single argument, `row`, which is a tuple containing the data for a single row.
# The first element of the tuple is the id, the second is the name, and the third is the breed.
# The method creates a new Dog instance with the name and breed from the row,
# and sets the id attribute to the value of the id column in the row.
# The new Dog instance is returned by the method.

@classmethod
def new_from_db(cls, row):
    id, name, breed = row
    dog = cls(name, breed)
    dog.id = id
    return dog

In [16]:
@classmethod
def get_all(cls):
    CURSOR.execute("SELECT * FROM dogs")
    rows = CURSOR.fetchall()
    dogs = []
    for row in rows:
        dog = Dog.new_from_db(row)
        dogs.append(dog)
    return dogs

In [ ]:
@classmethod
def find_by_name(cls, name):
    CURSOR.execute('SELECT * FROM dogs WHERE name = ?', (name,))
    row = CURSOR.fetchone()
    if row:
        return cls.new_from_db(row)
    else:
        return None

In [ ]:
@classmethod
def find_by_id(cls, id):
    CURSOR.execute('SELECT * FROM dogs WHERE id=?', (id,))
    result = CURSOR.fetchone()
    if result:
        return cls.new_from_db(result)

In [ ]:
@classmethod
def find_or_create_by(cls, name, breed):
    
    CURSOR.execute('SELECT * FROM dogs WHERE name=? AND breed=?', (name, breed))
    row = CURSOR.fetchone()
    if row:
     
        return cls.new_from_db(row)
    else:
       
        dog = cls(name, breed)
        dog.save()
        return dog

In [14]:
def update(self):
    if self.id is None:
        return

    CURSOR.execute('UPDATE dogs SET name = ?, breed = ? WHERE id = ?', (self.name, self.breed, self.id))
    CONN.commit()